In [1]:
import pandas as pd
from scipy.optimize import brentq
from math import log, sqrt, exp
from scipy.stats import norm
from datetime import datetime

# Load the Excel file
df=pd.read_excel("Walmart cleaned data.xlsx")

def bs_call_price(S, K, T, r, sigma):
    d1=(log(S/K)+(r+0.5*sigma**2)*T)/(sigma*sqrt(T))
    d2=d1-sigma*sqrt(T)
    return S*norm.cdf(d1)-K*exp(-r*T)*norm.cdf(d2)

def bs_put_price(S, K, T, r, sigma):
    d1=(log(S/K)+(r+0.5*sigma**2)*T)/(sigma*sqrt(T))
    d2=d1-sigma*sqrt(T)
    return K*exp(-r*T)*norm.cdf(-d2) - S * norm.cdf(-d1)

def implied_volatility_call(market_price, S, K, T, r):
    try:
        return brentq(lambda sigma: bs_call_price(S, K, T, r, sigma)-market_price, 1e-6, 5)
    except ValueError:
        return None

def implied_volatility_put(market_price, S, K, T, r):
    try:
        return brentq(lambda sigma: bs_put_price(S, K, T, r, sigma)-market_price, 1e-6, 5)
    except ValueError:
        return None

def time_to_maturity(expiration_date_str, current_date_str):
    expiration_date=datetime.strptime(expiration_date_str, "%m/%d/%y")
    current_date=datetime.strptime(current_date_str, "%m/%d/%y")
    return (expiration_date-current_date).days/365

df["Call_Mid"]=(df["Call_Bid"]+df["Call_Ask"])/2
df["Put_Mid"]=(df["Put_Bid"]+df["Put_Ask"])/2

call_iv_list=[]
put_iv_list=[]

for idx, row in df.iterrows():
    date_str=row["Date"].strftime("%m/%d/%y")
    current_date="12/13/24" if date_str=="12/20/24" else "12/20/24"

    T=time_to_maturity(date_str, current_date)
    S=row["Spot"]
    K=row["Strike"]
    r=row["Rf"]/100
    call_mid=row["Call_Mid"]
    put_mid=row["Put_Mid"]

    call_iv=implied_volatility_call(call_mid, S, K, T, r)
    put_iv=implied_volatility_put(put_mid, S, K, T, r)

    call_iv_list.append(call_iv)
    put_iv_list.append(put_iv)

df["ImpliedVol_Call"]=call_iv_list
df["ImpliedVol_Put"]=put_iv_list

output_file="Walmart_with_CallPut_IV.xlsx"
df.to_excel(output_file, index=False)
print(f"Exported call and put IVs to: {output_file}")


Exported call and put IVs to: Walmart_with_CallPut_IV.xlsx


In [2]:
import pandas as pd
from scipy.optimize import brentq
from math import log, sqrt, exp
from scipy.stats import norm
from datetime import datetime

df = pd.read_excel("AAPL Data.xlsx")

def bs_call_price(S, K, T, r, sigma):
    d1=(log(S/K)+(r+0.5*sigma**2)*T)/(sigma*sqrt(T))
    d2=d1-sigma*sqrt(T)
    return S*norm.cdf(d1)-K*exp(-r*T)*norm.cdf(d2)

def bs_put_price(S, K, T, r, sigma):
    d1=(log(S/K)+(r+0.5*sigma**2)*T)/(sigma*sqrt(T))
    d2=d1-sigma*sqrt(T)
    return K*exp(-r*T)*norm.cdf(-d2)-S*norm.cdf(-d1)

def implied_volatility_call(market_price, S, K, T, r):
    try:
        return brentq(lambda sigma: bs_call_price(S, K, T, r, sigma)-market_price, 1e-6, 5)
    except ValueError:
        return None

def implied_volatility_put(market_price, S, K, T, r):
    try:
        return brentq(lambda sigma: bs_put_price(S, K, T, r, sigma)-market_price, 1e-6, 5)
    except ValueError:
        return None

def time_to_maturity(expiration_date_str, current_date_str):
    expiration_date=datetime.strptime(expiration_date_str, "%m/%d/%y")
    current_date=datetime.strptime(current_date_str, "%m/%d/%y")
    return (expiration_date-current_date).days/365

df["Call_Mid"]=(df["Call_Bid"]+df["Call_Ask"])/2
df["Put_Mid"]=(df["Put_Bid"]+df["Put_Ask"])/2

iv_call_list=[]
iv_put_list=[]

for _, row in df.iterrows():
    date_str = row["Date"].strftime("%m/%d/%y")
    current_date=(datetime.strptime(date_str, "%m/%d/%y") - pd.Timedelta(days=7)).strftime("%m/%d/%y")
    
    T=time_to_maturity(date_str, current_date)
    S=row["Spot"]
    K=row["Strike"]
    r=row["Rf"] / 100
    call_mid=row["Call_Mid"]
    put_mid=row["Put_Mid"]
    
    iv_call=implied_volatility_call(call_mid, S, K, T, r)
    iv_put=implied_volatility_put(put_mid, S, K, T, r)
    
    iv_call_list.append(iv_call)
    iv_put_list.append(iv_put)

df["ImpliedVol_Call"]=iv_call_list
df["ImpliedVol_Put"]=iv_put_list

output_filename="AAPL_with_CallPut_IV.xlsx"
df.to_excel(output_filename, index=False)
print(f"File saved: {output_filename}")


File saved: AAPL_with_CallPut_IV.xlsx
